In [2]:
import logging

from learn_ai.scripts.utils import chat_completion_request_tokens

In [7]:
# Define the cost per million tokens for each model
COST_MILLION_TOKENS = {
    "gpt-4": {"input": 0.02, "output": 0.06},
    "gpt-4-0125-preview": {"input": 0.02, "output": 0.06},
    "gpt-4-1106-preview": {"input": 0.02, "output": 0.06},
    "gpt-4-1106-vision-preview": {"input": 0.02, "output": 0.06},
    "gpt-3.5-turbo-0613": {"input": 0.01, "output": 0.02},
    "gpt-3.5-turbo-0125": {"input": 0.01, "output": 0.02},
    "gpt-3.5-turbo-instruct": {"input": 0.01, "output": 0.02}
}

# Calculate cost based on token counts and model type
def calculate_cost(total_tokens, input_tokens, output_tokens, model_type):
    input_cost = (input_tokens / 1e6) * COST_MILLION_TOKENS[model_type]['input']
    output_cost = (output_tokens / 1e6) * COST_MILLION_TOKENS[model_type]['output']
    total_cost = input_cost + output_cost
    return total_cost, input_cost, output_cost

# Update and log costs after each chat interaction
class CostLogger:
    def __init__(self):
        self.costs = {"total_cost": 0, "total_input_cost": 0, "total_output_cost": 0, "costs_per_model": {}}
        self.token_count = {"total_token": 0, "total_input_token": 0, "total_output_token": 0, "token_per_model": {}}
        
    def update_cost_after_chat(self, tokens, model_type):
        total_tokens, input_tokens, output_tokens = tokens
        total_cost, input_cost, output_cost = calculate_cost(total_tokens, input_tokens, output_tokens, model_type)
        self.costs["total_cost"] += total_cost
        self.costs["total_input_cost"] += input_cost
        self.costs["total_output_cost"] += output_cost
        self.costs.setdefault("costs_per_model", {}).setdefault(model_type, {"total": 0, "input": 0, "output": 0})
        self.costs["costs_per_model"][model_type]["total"] += total_cost
        self.costs["costs_per_model"][model_type]["input"] += input_cost
        self.costs["costs_per_model"][model_type]["output"] += output_cost
        logging.info(self.costs)

    def log_costs(self):
        print(f"Total cost for conversation: ${self.costs['total_cost']:.8f}")
        print(f"Total input cost: ${self.costs['total_input_cost']:.8f}")
        print(f"Total output cost: ${self.costs['total_output_cost']:.8f}")
        for model, costs in self.costs["costs_per_model"].items():
            print(f"Costs for {model}: Total: ${costs['total']:.8f}, Input: ${costs['input']:.8f}, Output: ${costs['output']:.8f}")

    def update_tokens_after_chat(self, tokens, model_type):
        total_tokens, input_tokens, output_tokens = tokens
        self.token_count["total_token"] += total_tokens
        self.token_count["total_input_token"] += input_tokens
        self.token_count["total_output_token"] += output_tokens
        self.token_count.setdefault("token_per_model", {}).setdefault(model_type, {"input": 0, "output": 0})
        self.token_count["token_per_model"][model_type]["input"] += input_tokens
        self.token_count["token_per_model"][model_type]["output"] += output_tokens
        logging.info(f"Updated token counts after chat for model {model_type}: {self.token_count}")

    def log_tokens(self):
        print(f"Total tokens for conversation: {self.token_count['total_token']}")
        print(f"Total input tokens: {self.token_count['total_input_token']}")
        print(f"Total output tokens: {self.token_count['total_output_token']}")
        for model, token_counts in self.token_count["token_per_model"].items():
            print(f"Tokens for {model}: Input: {token_counts['input']}, Output: {token_counts['output']}")


In [8]:
cost_logger = CostLogger()

# Simulate sending messages and getting a completion
model_type = 'gpt-3.5-turbo-0125'
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm good, thank you! How can I assist you today?"}
]

# Assume the function returns the completion and token counts
_, tokens = chat_completion_request_tokens(messages, model=model_type)

# Update and log the costs based on the tokens used
cost_logger.update_cost_after_chat(tokens, model_type)
cost_logger.log_costs()

# Log the token counts for teaching purposes
cost_logger.update_tokens_after_chat(tokens, model_type)
cost_logger.log_tokens()

INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:{'total_cost': 4.9e-07, 'total_input_cost': 3.1e-07, 'total_output_cost': 1.8e-07, 'costs_per_model': {'gpt-3.5-turbo-0125': {'total': 4.9e-07, 'input': 3.1e-07, 'output': 1.8e-07}}}
INFO:Updated token counts after chat for model gpt-3.5-turbo-0125: {'total_token': 40, 'total_input_token': 31, 'total_output_token': 9, 'token_per_model': {'gpt-3.5-turbo-0125': {'input': 31, 'output': 9}}}


Total cost for conversation: $0.00000049
Total input cost: $0.00000031
Total output cost: $0.00000018
Costs for gpt-3.5-turbo-0125: Total: $0.00000049, Input: $0.00000031, Output: $0.00000018
Total tokens for conversation: 40
Total input tokens: 31
Total output tokens: 9
Tokens for gpt-3.5-turbo-0125: Input: 31, Output: 9
